In [1]:
#r "nuget:Serilog.Formatting.Compact.Reader,1.0.3"
#r "nuget:System.Linq.Async,4.0.0"

In [1]:
using System.IO;
using Serilog.Events;
using Serilog.Formatting.Compact.Reader;
using System.Collections.Generic;
using System.Linq;

Installed package Serilog.Formatting.Compact.Reader version 1.0.3

Installed package System.Linq.Async version 4.0.0

In [1]:
public async IAsyncEnumerable<PlotDatum> GetBestFrameCount()
{
    var compactFile = new FileInfo(@"C:\Users\vaido\SourceCode\MrBabyData\FrameRateAnalysis\RogueTD_PerformanceMetricsFrameRates.clef");

    using(var stream = compactFile.OpenText())
    {
        while(stream.Peek() > 0){
            var logevent = Serilog.Formatting.Compact.Reader.LogEventReader.ReadFromString(await stream.ReadLineAsync());
          
            yield return new PlotDatum{
                timeStamp = logevent.Timestamp,
                bestFrameCount = float.Parse(logevent.Properties["bestDuration"].ToString())
            };
        }
    }
}

public struct PlotDatum{
    public DateTimeOffset timeStamp {get; set;}
    public float bestFrameCount {get; set;}
}

var plotDatums = await GetBestFrameCount().ToListAsync();

In [1]:
#!javascript
notebookScope.plot = (sgvSelector, variableName) => {
    
    // RequireJS is some kind of framework for loadin npm modules?
    let dtreeLoader = interactive.configureRequire({
        paths: {
            d3: "https://d3js.org/d3.v6.min"
        }
    });

    // not sure, but require JS needs us to run d3 code in this function?P
    dtreeLoader(["d3"], function (d3) {

    // Plot builder
    function buildFrameRatePlot(svg, width, height, data) {  
        // var some parameters for drawing this
                var margin = 16;
        
                // clean clear our svg
                svg.selectAll("g").remove();
        
                svg.append("rect")
                    .attr("height", height)
                    .attr("width", width)
                    .style("fill", "white")
        
                // plot container
                let container = svg
                    .append("g")
        
                // create my scales
                var y = d3.scaleLinear()
                    .domain([0, 120])
                    .range([height - margin, 0])
                var x = d3.scaleLinear()
                    .domain([0, 900])
                    .range([0, width - margin * 2])
        
            var yAxis = g => g
                .attr("transform", `translate(30,8)`)
                .call(d3.axisLeft(y))
            var xAxis = g => g
                .attr("transform", `translate(30, ${height - 8})`)
                .call(d3.axisBottom(x))
        
            svg.append("g")
                .call(yAxis);
            svg.append("g")
                .call(xAxis);
        }
        
        // var some parameters for drawing this
        var height = 300;
        var width = 1000;

        // get a reference to our svg
        var svg = d3.
            select(sgvSelector)

        buildFrameRatePlot(svg, width, height)

        return svg.node()
    });
}

In [1]:
#!html
<svg id="dataPlot1" height=300 width=1000></svg>

#!js
notebookScope.plot("svg#dataPlot1", "plotDatums");